# Imports

In [1]:
import pandas as pd
import numpy as np

---

# Carga de datos

In [3]:
auctions = pd.read_pickle("../../../data/tp2/auctions_tp2_formateado.pkl")

---

# Armado de ventanas

In [48]:
for ventana_nro in range(1,6):
    auctions["ventana_{}".format(ventana_nro)] = (((17+ventana_nro) <= auctions["date"].dt.day) & ((auctions["date"].dt.day) <= (19+ventana_nro)))

---

# Armado de validaciones

In [49]:
for validacion_nro in range(1,6):
    auctions["validacion_{}".format(validacion_nro)] = (((20+validacion_nro) <= auctions["date"].dt.day) & ((auctions["date"].dt.day) <= (22+validacion_nro)))

---

# Obtención del tiempo hasta aparición ("objetivo" o "label" o "target") para cada ventana de datos

In [136]:
tres_dias = 3*24*60*60
for validacion_nro in range(1,6):
    validacion = auctions.loc[auctions["validacion_{}".format(validacion_nro)]].groupby("device_id").agg({"date" : "min"}).reset_index()
    validacion["target"] = validacion["date"] - np.datetime64('2019-04-{} 00:00:00.000000'.format(20+validacion_nro))
    del validacion["date"]
    validacion["target"] = validacion["target"].map(lambda x: (x.delta / 1000000000))
    target = auctions.loc[auctions["ventana_{}".format(validacion_nro)]]["device_id"].drop_duplicates().to_frame()
    target = target.reset_index()
    del target["index"]
    target = target.merge(validacion_1, on="device_id", how="left").fillna(tres_dias)
    target.to_pickle("../targets/targets_ventana_{}_auctions.pkl".format(validacion_nro))

# Razonamiento para obtener el delirio de arriba

In [109]:
validacion_1 = auctions.loc[auctions["validacion_1"]].groupby("device_id").agg({"date" : "min"}).reset_index()

In [110]:
validacion_1["target"] = validacion_1["date"] - np.datetime64('2019-04-21 00:00:00.000000')

In [111]:
del validacion_1["date"]

In [112]:
validacion_1["target"] = validacion_1["target"].map(lambda x: (x.delta / 1000000000))

In [122]:
target = auctions.loc[auctions["ventana_1"]]["device_id"].drop_duplicates().to_frame()

In [124]:
target = target.reset_index()

In [125]:
del target["index"]

Luego se hace el merge para quedarme con los datos que estan en la ventana